In [7]:
import argparse
import sys
import time
import numpy as np

In [2]:
def _parse_args():
    parser = argparse.ArgumentParser(description='trainer.py')
    parser.add_argument('--model', type=str, default='BAD', help='model to run (BAD, CLASSIFIER)')
    parser.add_argument('--train_path', type=str, default='data/eng.train', help='path to train set (you should not need to modify)')
    parser.add_argument('--dev_path', type=str, default='data/eng.testa', help='path to dev set (you should not need to modify)')
    parser.add_argument('--blind_test_path', type=str, default='data/eng.testb.blind', help='path to dev set (you should not need to modify)')
    parser.add_argument('--test_output_path', type=str, default='eng.testb.out', help='output path for test predictions')
    parser.add_argument('--no_run_on_test', dest='run_on_test', default=True, action='store_false', help='skip printing output on the test set')
    args = parser.parse_args()
    return args


# Wrapper for an example of the person binary classification task.
# tokens: list of string words
# labels: list of (0, 1) where 0 is non-name, 1 is name
class PersonExample(object):
    def __init__(self, tokens, labels):
        self.tokens = tokens
        self.labels = labels

    def __len__(self):
        return len(self.tokens)


# Changes NER-style chunk examples into binary classification examples.
def transform_for_classification(ner_exs):
    # Take each LabeledSentence object and extract the bio tags.
    for labeled_sent in ner_exs:
        tags = bio_tags_from_chunks(labeled_sent.chunks, len(labeled_sent))

        # create a list "labels" that has 1 for every position with a person's name, 0 otherwise
        labels = [1 if tag.endswith("PER") else 0 for tag in tags]

        # Yield a PersonExample object with a list of tokens and labels
        yield PersonExample([tok.word for tok in labeled_sent.tokens], labels)

In [3]:
class CountBasedPersonClassifier(object):
    def __init__(self, pos_counts, neg_counts):
        self.pos_counts = pos_counts
        self.neg_counts = neg_counts

    def predict(self, tokens, idx):
        # simply checks that the "positive" word count is higher than the "negative" word count in a sentence
        if self.pos_counts.get_count(tokens[idx]) > self.neg_counts.get_count(tokens[idx]):
            return 1
        else:
            return 0

In [4]:

# "Trains" the count-based person classifier by collecting counts over the given examples.
def train_count_based_binary_classifier(ner_exs):
    pos_counts = Counter()
    neg_counts = Counter()
    for ex in ner_exs:
        for idx in range(0, len(ex)):
            if ex.labels[idx] == 1:
                pos_counts.increment_count(ex.tokens[idx], 1.0)
            else:
                neg_counts.increment_count(ex.tokens[idx], 1.0)
    return CountBasedPersonClassifier(pos_counts, neg_counts)

In [5]:
def evaluate_classifier(exs, classifier):
    num_correct = 0
    num_pos_correct = 0
    num_pred = 0
    num_gold = 0
    num_total = 0
    for ex in exs:
        for idx in range(0, len(ex)):
            prediction = classifier.predict(ex.tokens, idx)
            if prediction == ex.labels[idx]:
                num_correct += 1
            if prediction == 1:
                num_pred += 1
            if ex.labels[idx] == 1:
                num_gold += 1
            if prediction == 1 and ex.labels[idx] == 1:
                num_pos_correct += 1
            num_total += 1
    print("Accuracy: %i / %i = %f" % (num_correct, num_total, float(num_correct) / num_total))
    prec = float(num_pos_correct) / num_pred if num_pred > 0 else 0.0
    rec = float(num_pos_correct) / num_gold if num_gold > 0 else 0.0
    f1 = 2 * prec * rec/(prec + rec) if prec > 0 and rec > 0 else 0.0
    print("Precision: %i / %i = %f" % (num_pos_correct, num_pred, prec))
    print("Recall: %i / %i = %f" % (num_pos_correct, num_gold, rec))
    print("F1: %f" % f1)


In [6]:

# Runs prediction on exs and writes the outputs to outfile, one token per line
def predict_write_output_to_file(exs, classifier, outfile):
    f = open(outfile, 'w')
    for ex in exs:
        for idx in range(0, len(ex)):
            prediction = classifier.predict(ex.tokens, idx)
            f.write(ex.tokens[idx] + " " + repr(int(prediction)) + "\n")
        f.write("\n")
    f.close()


In [8]:
# nerdata.py


# Abstraction to bundle words with POS and chunks for featurization
class Token:
    def __init__(self, word, pos, chunk):
        self.word = word
        self.pos = pos
        self.chunk = chunk

    def __repr__(self):
        return "Token(%s, %s, %s)" % (self.word, self.pos, self.chunk)

    def __str__(self):
        return self.__repr__()


# Thin wrapper around a start and end index coupled with a label, representing,
# e.g., a chunk PER over the span (3,5). Indices are semi-inclusive, so (3,5)
# contains tokens 3 and 4 (0-based indexing).
class Chunk:
    def __init__(self, start_idx, end_idx, label):
        self.start_idx = start_idx
        self.end_idx = end_idx
        self.label = label

    def __repr__(self):
        return "(" + repr(self.start_idx) + ", " + repr(self.end_idx) + ", " + self.label + ")"

    def __str__(self):
        return self.__repr__()

    def __eq__(self, other):
        if isinstance(other, self.__class__):
            return self.start_idx == other.start_idx and self.end_idx == other.end_idx and self.label == other.label
        else:
            return False

    def __ne__(self, other):
        return not self.__eq__(other)

    def __hash__(self):
        return hash(self.start_idx) + hash(self.end_idx) + hash(self.label)


# Thin wrapper over a sequence of Tokens representing a sentence and an optional set of chunks
# representation NER labels, which are also stored as BIO tags
class LabeledSentence:
    def __init__(self, tokens, chunks=None):
        self.tokens = tokens
        self.chunks = chunks
        if chunks is None:
            self.bio_tags = None
        else:
            self.bio_tags = bio_tags_from_chunks(self.chunks, len(self.tokens))

    def __repr__(self):
        return repr([repr(tok) for tok in self.tokens]) + "\n" + repr([repr(chunk) for chunk in self.chunks])

    def __str__(self):
        return self.__repr__()

    def __len__(self):
        return len(self.tokens)

    def get_bio_tags(self):
        return self.bio_tags


# We store NER tags as strings, but they contain two pieces:
# a coarse tag type (BIO) and a label (PER), e.g. B-PER
def isB(ner_tag):
    return ner_tag.startswith("B")


def isI(ner_tag):
    return ner_tag.startswith("I")


def isO(ner_tag):
    return ner_tag == "O"


# Gets the label component of the NER tag: e.g., returns PER for B-PER
def get_tag_label(ner_tag):
    if len(ner_tag) > 2:
        return ner_tag[2:]
    else:
        return None


# Convert BIO tags to (start, end, label) chunk representations
# (start, end) are semi-inclusive, meaning that in the sentence
# He met Barack Obama yesterday
# Barack Obama has the span (2, 4)
# N.B. this method only works because chunks are non-overlapping in this data
def chunks_from_bio_tag_seq(bio_tags):
    chunks = []
    curr_tok_start = -1
    curr_tok_label = ""
    for idx, tag in enumerate(bio_tags):
        if isB(tag):
            label = get_tag_label(tag)
            if curr_tok_label != "":
                chunks.append(Chunk(curr_tok_start, idx, curr_tok_label))
            curr_tok_label = label
            curr_tok_start = idx
        elif isI(tag):
            label = get_tag_label(tag)
            if label != curr_tok_label:
                print("WARNING: invalid tag sequence (I after O); ignoring the I: %s" % bio_tags)
        else: # isO(tag):
            if curr_tok_label != "":
                chunks.append(Chunk(curr_tok_start, idx, curr_tok_label))
            curr_tok_label = ""
            curr_tok_start = -1
    return chunks


# Converts a chunk representation back to BIO tags
def bio_tags_from_chunks(chunks, sent_len):
    tags = []
    for i in range(0, sent_len):
        matching_chunks = list(filter(lambda chunk: chunk.start_idx <= i and i < chunk.end_idx, chunks))
        if len(matching_chunks) > 0:
            if i == matching_chunks[0].start_idx:
                tags.append("B-" + matching_chunks[0].label)
            else:
                tags.append("I-" + matching_chunks[0].label)
        else:
            tags.append("O")
    return tags


# Reads a dataset in the CoNLL format from a file
# The format is one token per line:
# [word] [POS] [syntactic chunk] *potential junk column* [NER tag]
# One blank line appears after each sentence
def read_data(file):
    f = open(file)
    sentences = []
    curr_tokens = []
    curr_bio_tags = []
    for line in f:
        stripped = line.strip()
        if stripped != "":
            # split the line into tokens
            fields = stripped.split(" ")

            # If fields contains 4 or 5 "words", then append the first three to curr_tokens,
            # and append the last to curr_bio_tags
            if len(fields) == 4 or len(fields) == 5:
                curr_tokens.append(Token(fields[0], fields[1], fields[2]))
                curr_bio_tags.append(fields[-1])

        # If the line is empty, then the sentence is complete. Create a LabeledSentence object and
        # add it to the list "sentences"
        elif stripped == "" and len(curr_tokens) > 0:
            sentences.append(LabeledSentence(curr_tokens, chunks_from_bio_tag_seq(curr_bio_tags)))
            curr_tokens = []
            curr_bio_tags = []

    # return the list of LabeledSentence objects
    return sentences


# Evaluates the guess sentences with respect to the gold sentences
def print_evaluation(gold_sentences, guess_sentences):
    correct = 0
    num_pred = 0
    num_gold = 0
    for gold, guess in zip(gold_sentences, guess_sentences):
        correct += len(set(guess.chunks) & set(gold.chunks))
        num_pred += len(guess.chunks)
        num_gold += len(gold.chunks)
    if num_pred == 0:
        prec = 0
    else:
        prec = correct/float(num_pred)
    if num_gold == 0:
        rec = 0
    else:
        rec = correct/float(num_gold)
    if prec == 0 and rec == 0:
        f1 = 0
    else:
        f1 = 2 * prec * rec / (prec + rec)
    print("Labeled F1: " + "{0:.2f}".format(f1 * 100) +\
          ", precision: %i/%i" % (correct, num_pred) + " = " + "{0:.2f}".format(prec * 100) + \
          ", recall: %i/%i" % (correct, num_gold) + " = " + "{0:.2f}".format(rec * 100))


# Writes labeled_sentences to outfile in the CoNLL format
def print_output(labeled_sentences, outfile):
    f = open(outfile, 'w')
    for sentence in labeled_sentences:
        bio_tags = sentence.get_bio_tags()
        for i in range(0, len(sentence)):
            tok = sentence.tokens[i]
            f.write(tok.word + " " + tok.pos + " " + tok.chunk + " " + bio_tags[i] + "\n")
        f.write("\n")
    print("Wrote predictions on %i labeled sentences to %s" % (len(labeled_sentences), outfile))
    f.close()

In [9]:
# adagrad_trainer.py

from utils import *
from abc import ABC, abstractmethod
import numpy as np


class Optimizer(ABC):
    # Scores a sparse feature vector
    # feats: list of integer feature indices
    def score(self, feats):
        i = 0
        score = 0.0
        while i < len(feats):
            score += self.access(feats[i])
            i += 1
        return score

    @abstractmethod
    def apply_gradient_update(self, gradient, batch_size):
        pass

    @abstractmethod
    def access(self, i):
        pass

    @abstractmethod
    def get_final_weights(self, i):
        pass


# SGD optimizer implementation, designed to have the same interface as the Adagrad optimizers
class SGDOptimizer(Optimizer):
    # init_weights: a numpy array of the correct dimension, usually initialized to 0
    # alpha: step size
    def __init__(self, init_weights, alpha):
        self.weights = init_weights
        self.alpha = alpha

    def apply_gradient_update(self, gradient, batch_size):
        for i in gradient.keys():
            g = gradient.get_count(i)
            self.weights[i] = self.weights[i] + self.alpha * g

    # Get the weight of feature i
    def access(self, i):
        return self.weights[i]

    def get_final_weights(self):
        return self.weights


# Wraps a weight vector and applies the Adagrad update using second moments of features to make custom step sizes.
# This version incorporates L1 regularization: while this regularization should be applied to squash the feature vector
# on every gradient update, we instead evaluate the regularizer lazily only when the particular feature is touched
# (either by gradient update or by access). approximate lets you turn this off for faster access, but regularization is
# now applied somewhat inconsistently.
# See section 5.1 of http://www.jmlr.org/papers/volume12/duchi11a/duchi11a.pdf for more details
class L1RegularizedAdagradTrainer(Optimizer):
    # init_weights: a numpy array of the correct dimension, usually initialized to 0
    # lamb: float lambda constant for the regularizer. Values above 0.01 will often cause all features to be zeroed out.
    # eta: float step size. Values from 0.01 to 10 often work well.
    # approximate: turns off gradient updates on access, only uses them when weights are written to.
    # So regularization is applied inconsistently, but it makes things faster.
    def __init__(self, init_weights, lamb=1e-8, eta=1.0, use_regularization=False, approximate=True):
        self.weights = init_weights
        self.lamb = lamb
        self.eta = eta
        self.use_regularization = use_regularization
        self.approximate = approximate
        self.curr_iter = 0
        self.last_iter_touched = [0 for i in range(0, self.weights.shape[0])]
        self.diag_Gt = np.zeros_like(self.weights, dtype=float)

    # Take a sparse representation of the gradient and make an update, normalizing by the batch size to keep
    # hyperparameters constant as the batch size is varied
    # gradient: Counter
    # batch_size: integer
    def apply_gradient_update(self, gradient, batch_size):
        batch_size_multiplier = 1.0 / batch_size
        self.curr_iter += 1
        for i in gradient.keys():
            xti = self.weights[i]
            # N.B.We negate the gradient here because the Adagrad formulas are all for minimizing
            # and we're trying to maximize, so think of it as minimizing the negative of the objective
            # which has the opposite gradient
            # See section 5.1 in http://www.jmlr.org/papers/volume12/duchi11a/duchi11a.pdf for more details
            # eta is the step size, lambda is the regularization
            gti = -gradient.get_count(i) * batch_size_multiplier
            old_eta_over_Htii = self.eta / (1 + np.sqrt(self.diag_Gt[i]))
            self.diag_Gt[i] += gti * gti
            Htii = 1 + np.sqrt(self.diag_Gt[i])
            eta_over_Htii = self.eta / Htii
            new_xti = xti - eta_over_Htii * gti
            # Apply the regularizer for every iteration since touched
            iters_since_touched = self.curr_iter - self.last_iter_touched[i]
            self.last_iter_touched[i] = self.curr_iter
            self.weights[i] = np.sign(new_xti) * max(0, np.abs(new_xti) - self.lamb * eta_over_Htii - (iters_since_touched - 1) * self.lamb * old_eta_over_Htii)

    # Get the weight of feature i
    def access(self, i):
        if not self.approximate and self.last_iter_touched[i] != self.curr_iter:
            xti = self.weights[i]
            Htii = 1 + np.sqrt(self.diag_Gt[i])
            eta_over_Htii = self.eta / Htii
            iters_since_touched = self.curr_iter - self.last_iter_touched[i]
            self.last_iter_touched[i] = self.curr_iter
            self.weights[i] = np.sign(xti) * max(0, np.abs(xti) - iters_since_touched * self.lamb * self.eta * eta_over_Htii);
        return self.weights[i]

    # Return a numpy array containing the final weight vector values -- manually calls access to force each weight to
    # have an updated value.
    def get_final_weights(self):
        for i in range(0, self.weights.shape[0]):
            self.access(i)
        return self.weights


# Applies the Adagrad update with no regularization. Will be substantially faster than the L1 regularized version
# due to less computation required to update each feature.
class UnregularizedAdagradTrainer(Optimizer):
    def __init__(self, init_weights, eta=1.0):
        self.weights = init_weights
        self.eta = eta
        self.diag_Gt = np.zeros_like(self.weights, dtype=float)

    def apply_gradient_update(self, gradient, batch_size):
        batch_size_multiplier = 1.0 / batch_size
        for i in gradient.keys():
            xti = self.weights[i]
            gti = -gradient.get_count(i) * batch_size_multiplier
            self.diag_Gt[i] += gti * gti
            Htii = 1 + np.sqrt(self.diag_Gt[i])
            eta_over_Htii = self.eta / Htii
            self.weights[i] = xti - eta_over_Htii * gti

    # Get the weight of feature i
    def access(self, i):
        return self.weights[i]

    # Return a numpy array containing the final weight vector values -- manually calls access to force each weight to
    # have an updated value.
    def get_final_weights(self):
        return self.weights

In [10]:
class PersonClassifier(object):
    def __init__(self, weights, indexer):
        self.weights = weights
        self.indexer = indexer

    # Makes a prediction for token at position idx in the given PersonExample
    def predict(self, tokens, idx):
        raise Exception("Implement me!")

In [11]:
def train_classifier(ner_exs):
    # Todo: Implement a training method here, follow steps below:
    # use counter to keep track of gradient
    # can do vector implementation, using dot product instead of looping over each element

    # Probably want to implement a sigmoid (logistic regression) classifier here.
    raise Exception("Implement me!")
    # Will need to calculate the gradient as well

In [12]:
def sigmoid(weights, inputs):
    """Implement logistic regression here. Takes two numpy arrays, calculates their dot product,
    and plugs it into sigmoid formula"""
    z = np.dot(weights, inputs)
    out = 1/(1+exp(z))

In [13]:
def main():
    start_time = time.time()    # saves start time for calculation of running time
    args = _parse_args()    # _parse_args() uses argparse to determine constraints (such as model to run)
    print(args)

    # Load the training and test data
    train_class_exs = list(transform_for_classification(read_data(args.train_path)))
    dev_class_exs = list(transform_for_classification(read_data(args.dev_path)))

    # Train the model
    if args.model == "BAD":
        classifier = train_count_based_binary_classifier(train_class_exs)
    else:
        classifier = train_classifier(train_class_exs)


    print("Data reading and training took %f seconds" % (time.time() - start_time))
    # Evaluate on training, development, and test data
    print("===Train accuracy===")
    evaluate_classifier(train_class_exs, classifier)
    print("===Dev accuracy===")
    evaluate_classifier(dev_class_exs, classifier)
    if args.run_on_test:
        print("Running on test")
        test_exs = list(transform_for_classification(read_data(args.blind_test_path)))
        predict_write_output_to_file(test_exs, classifier, args.test_output_path)
        print("Wrote predictions on %i labeled sentences to %s" % (len(test_exs), args.test_output_path))



In [15]:
main()

usage: ipykernel_launcher.py [-h] [--model MODEL] [--train_path TRAIN_PATH]
                             [--dev_path DEV_PATH]
                             [--blind_test_path BLIND_TEST_PATH]
                             [--test_output_path TEST_OUTPUT_PATH]
                             [--no_run_on_test]
ipykernel_launcher.py: error: unrecognized arguments: -f C:\Users\rwest\AppData\Roaming\jupyter\runtime\kernel-67fc0dc0-9feb-4c74-a21b-6886fc9e0706.json


SystemExit: 2

c:\users\rwest\.virtualenvs\mini1-v-kswoty\lib\site-packages\IPython\core\interactiveshell.py:2969: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
